In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as units

In [ ]:
%autoreload 2
import madart.dualpy as dp

In [ ]:
def func(x,p):
    return (x+p)**2

In [ ]:
x_true = np.arange(1,11) * units.m
p = 0.0*units.m
p_true = p
q = 0.0*units.m**2
q_true = q

In [ ]:
p = dp._seed_dense(p, "p")
q = dp._seed_dense(q, "q")

In [ ]:
y_target = func(x_true, p_true) + q

In [ ]:
print(x_true)
print(y_target)
print(dp.has_jacobians(y_target))

In [ ]:
x0 = x_true*0.0+1.0*units.m
x_hat = dp.multi_newton_raphson(x0, func, args=[p], y=y_target, max_iter=None, dx_tolerance=1e-12*units.m)

In [ ]:
print(x_hat-x_true)

In [ ]:
print(x_hat.jacobians)

In [ ]:
jp = x_hat.jacobians["p"].todensearray()
jq = x_hat.jacobians["q"].todensearray()
jq_test = 0.5*(y_target)**(-0.5)

In [ ]:
print(jp)
print(jq)
print(jq_test)

In [ ]:
print(x_hat.to(units.km))